In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
dname= 'Heartbeat'
file_path = f'./datasets/{dname}/{dname}_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


(405, 204) (204,)


In [2]:
file_path = f'./datasets/{dname}/{dname}_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

(405, 205) (205,)


In [3]:
X_train  = X.transpose()

In [4]:
X_test = X_test.transpose()

In [19]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y = label_encoder.fit_transform(y)
y_test = label_encoder.fit_transform(y_test)


In [20]:
num_classes = len(np.unique(y_test))

num_classes

2

In [25]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'binary',  # or 'multiclass' for multi-class classification
        'metric': 'binary_logloss',  # or 'multi_logloss' for multi-class classification

        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [26]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:35:36,550] A new study created in memory with name: no-name-0561413e-3bbb-4f22-a1f6-ef370dab4ae6
[I 2024-02-08 17:35:36,648] Trial 0 finished with value: 0.7219512195121951 and parameters: {'num_leaves': 195, 'learning_rate': 3.6524505113959014e-06, 'n_estimators': 94}. Best is trial 0 with value: 0.7219512195121951.
[I 2024-02-08 17:35:36,774] Trial 1 finished with value: 0.7219512195121951 and parameters: {'num_leaves': 75, 'learning_rate': 0.018094979807040723, 'n_estimators': 293}. Best is trial 0 with value: 0.7219512195121951.
[I 2024-02-08 17:35:36,961] Trial 2 finished with value: 0.7219512195121951 and parameters: {'num_leaves': 162, 'learning_rate': 8.040625550619272e-07, 'n_estimators': 385}. Best is trial 0 with value: 0.7219512195121951.
[I 2024-02-08 17:35:37,079] Trial 3 finished with value: 0.7219512195121951 and parameters: {'num_leaves': 247, 'learning_rate': 1.444303038051433e-07, 'n_estimators': 155}. Best is trial 0 with value: 0.7219512195121951.


Number of finished trials: 100
Best trial: {'num_leaves': 54, 'learning_rate': 0.0038855632299937878, 'n_estimators': 725}
